In [1]:
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )
import json
import requests
import pandas as pd
from decimal import Decimal

Carbon Version v2.4.1-BETA1 (17/Mar/2023)


# Carbon Simulation - Demo TestSuite

This book is for creating test scenarios for Tenderly Contracts

#### Basic token getter for decimals

In [2]:
r = requests.get('https://tokens.coingecko.com/uniswap/all.json')
cgtokens = pd.DataFrame(eval(r.text)['tokens'])

tokenDecimals = {}
tokenDecimals['ETH'] = 18
for i in cgtokens.index:
    tokenDecimals[cgtokens.symbol[i]] = int(cgtokens.decimals[i])

## Enter tkn0 and tkn1

In [7]:
tkn0 = "ETH"  #base token
tkn1 = "USDC" #quote token

token0Decimals = tokenDecimals[tkn0]
token1Decimals = tokenDecimals[tkn1]
print(tkn0, token0Decimals)
print(tkn1, token1Decimals)

ETH 18
USDC 6


## Initialize Simulator instance

In [8]:
Sim = CarbonSimulatorUI(pair=f"{tkn0}/{tkn1}", verbose=False, matching_method='fast')  # important to denote fast matching. pairs are specified base/quote and strategy is entered in quote per base units
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETH/USDC', mm='fast', xf=True)

## Add strategies

In [9]:
for i in range(2):
    Sim.add_strategy(
        tkn=f'{tkn0}',          # specify sell side tkn
        amt_sell=20,            # liqudity amount of tkn to sell
        psell_start=2000,       # price to start selling in quote per base units
        psell_end=2500,         # price to stop selling in quote per base units
        amt_buy=10000,          # liqudity amount of paired tkn to buy
        pbuy_start=1700,        # price to start buying in quote per base units
        pbuy_end=1000,          # price to stop buying in quote per base units
        pair=f"{tkn0}/{tkn1}"   # pair
                    interpret_decimals: bool = False,
            tkn_decimals: Any = None,
            tkn_other_decimals: Any = None,
    )['orders']

In [10]:
# for i in range(2):
#     Sim.add_strategy(
#         tkn=f'{tkn0}',          # specify sell side tkn
#         amt_sell=2_000_000_000,            # liqudity amount of tkn to sell
#         psell_start=4.7e-10,       # price to start selling in quote per base units
#         psell_end=5e-10,         # price to stop selling in quote per base units
#         amt_buy=1,          # liqudity amount of paired tkn to buy
#         pbuy_start=4.5e-10,        # price to start buying in quote per base units
#         pbuy_end=4e-10,          # price to stop buying in quote per base units
#         pair=f"{tkn0}/{tkn1}"   # pair
#     )['orders']

## Show all strategies

In [11]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,ETHUSDC,ETH,20.0,20.0,ETH,False,2000.0,2500.0,2000.0,USDC per ETH,1,
1,1,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1700.0,1000.0,1700.0,USDC per ETH,0,
2,2,ETHUSDC,ETH,20.0,20.0,ETH,False,2000.0,2500.0,2000.0,USDC per ETH,3,
3,3,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1700.0,1000.0,1700.0,USDC per ETH,2,


In [12]:
Sim.orders

{0: Order(B=0.02, S=0.0023606797749978967, y_int=Decimal('20'), p_high=Decimal('0.0005000000000000000104083408558608425664715468883514404296875'), p_low=Decimal('0.0004000000000000000191686944095437183932517655193805694580078125'), p_marginal=None, disabled=False, _y=Decimal('20'), D=Decimal('20'), _C=Decimal('20'), pair=P('ETH/USDC'), tkn='ETH', pair_name='ETHUSDC', adj_n_mode=False, auto_convert_variables=True, id=0, linked_to_id=1, _reverseq=False, verbose=False, carbon_id=''),
 1: Order(B=31.622776601683793, S=9.608279654492815, y_int=Decimal('10000'), p_high=Decimal('1700'), p_low=Decimal('1000'), p_marginal=None, disabled=False, _y=Decimal('10000'), D=Decimal('10000'), _C=Decimal('10000'), pair=P('ETH/USDC'), tkn='USDC', pair_name='ETHUSDC', adj_n_mode=False, auto_convert_variables=True, id=1, linked_to_id=0, _reverseq=False, verbose=False, carbon_id=''),
 2: Order(B=0.02, S=0.0023606797749978967, y_int=Decimal('20'), p_high=Decimal('0.00050000000000000001040834085586084256647154

## Do Trades in each direction
- Note that `execute=False` so each of these trades behave as if they are independent - i.e. the order balances do NOT change post-trade

## Enter an amount

In [98]:
amount = 0.000005

## APP - "Buy ETH with USDC" box

In [99]:
# Enter 1 USDC into the "BUY ETH for USDC" box
tbSou10 = Sim.amm_buys(f"{tkn1}", amount, execute=False, support_partial=False) # route_trade_by_source  #note the variable name "tbSou10" == {tradeBySource}{sourceToken}{targetToken}. e.g. in the TokensTraded event tkn1 should be the source token
tbSou10['trades']

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #0,False,False,False,None,10638.296148,SHIB,0.000005,WBTC,SHIBWBTC,0,1,4.700001e-10,WBTC per SHIB
0,1,1,A,AMM sells 10638SHIB buys 0WBTC,True,False,True,None,10638.296148,SHIB,0.000005,WBTC,SHIBWBTC,[0],1,4.700001e-10,WBTC per SHIB


In [102]:
amount = 500000000

In [103]:
# Enter 1 ETH into the "BUY ETH for USDC" box
tbTar10 = Sim.amm_sells(f"{tkn0}", amount, execute=False, support_partial=False) # route_trade_by_target
tbTar10['trades']

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,3.0,3,0,route #0,False,False,False,None,500000000.0,SHIB,0.236803,WBTC,SHIBWBTC,0,1,4.736070e-10,WBTC per SHIB
0,3,3,A,AMM sells 500000000SHIB buys 0WBTC,True,False,True,None,500000000.0,SHIB,0.236803,WBTC,SHIBWBTC,[0],1,4.736070e-10,WBTC per SHIB


## APP - "Sell ETH with USDC" box

In [111]:
amount = 1170038356

In [112]:
# Enter 1 ETH into the "SELL ETH for USDC" box
tbSou01 = Sim.amm_buys(f"{tkn0}", amount, execute=False, support_partial=False) # route_trade_by_source
tbSou01['trades']

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,8.0,8,0,route #1,False,False,False,None,0.511126,WBTC,1.170038e+09,SHIB,SHIBWBTC,1,1,4.368457e-10,WBTC per SHIB
0,8,8,A,AMM sells 1WBTC buys 1170038356SHIB,True,False,False,None,0.511126,WBTC,1.170038e+09,SHIB,SHIBWBTC,[1],1,4.368457e-10,WBTC per SHIB


In [110]:
# Enter 1 USDC into the "SELL ETH for USDC" box
tbTar01 = Sim.amm_sells(f"{tkn1}", amount, execute=False, support_partial=False) # route_trade_by_target
tbTar01['trades']

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,7.0,7,0,route #1,False,False,False,None,1.0,WBTC,2.357023e+09,SHIB,SHIBWBTC,1,1,4.242641e-10,WBTC per SHIB
0,7.1,7,1,route #3,False,False,False,None,1.0,WBTC,2.357023e+09,SHIB,SHIBWBTC,3,1,4.242641e-10,WBTC per SHIB
0,7,7,A,AMM sells 2WBTC buys 4714045208SHIB,True,False,True,None,2.0,WBTC,4.714045e+09,SHIB,SHIBWBTC,"[1, 3]",2,4.242641e-10,WBTC per SHIB


## Summary of trades

In [14]:
Sim.state()["trades"].query("subid=='A'")

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0,0,A,AMM sells 0ETH buys 85USDC,True,False,False,None,0.04249,ETH,85.000000,USDC,ETHUSDC,"[0, 2]",2,2000.448684,USDC per ETH
0,1,1,A,AMM sells 85ETH buys 189157USDC,True,False,True,None,85.00000,ETH,189156.525073,USDC,ETHUSDC,"[0, 2, 4, 6, 8]",5,2225.370883,USDC per ETH
0,2,2,A,AMM sells 85USDC buys 0ETH,True,False,True,None,85.00000,USDC,0.050099,ETH,ETHUSDC,[1],1,1696.632644,USDC per ETH
0,3,3,A,AMM sells 50000USDC buys 38ETH,True,False,True,None,50000.00000,USDC,38.348249,ETH,ETHUSDC,"[1, 3, 5, 7, 9]",5,1303.840481,USDC per ETH
